In [10]:
import gradio as gr
import re
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from transformers import pipeline
from langchain.llms import LlamaCpp
from huggingface_hub import hf_hub_download

# 1) Download your model
model_path = hf_hub_download(
    repo_id="TheBloke/Llama-2-7B-Chat-GGUF",
    filename="llama-2-7b-chat.Q4_0.gguf",
    local_dir="./models",
    local_dir_use_symlinks=False
)

# 2) Create LlamaCpp instance
llm = LlamaCpp(
    model_path=model_path,
    temperature=0.7,
    max_tokens=256,
    top_p=0.95,
    repeat_penalty=1.15,
    n_ctx=1024,
    n_gpu_layers=40,
    n_batch=512,
    verbose=True
)

# 3) Create your prompt + LLMChain
bot_prompt = PromptTemplate(
    input_variables=["user_message"],
    template=(
        "You are a helpful airline booking assistant for Fictitious Air.\n"
        "User: {user_message}\n"
        "Assistant:"
    ),
)
response_chain = LLMChain(llm=llm, prompt=bot_prompt)

# 4) Then proceed with the rest of your code
# (dummy_flights_data, process_user_input, etc.)
# ...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:834: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(
llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from models/llama-2-7b-chat.Q4_0.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_m

In [11]:
import gradio as gr
import re
from transformers import pipeline

###############################################################################
# 1) Dummy Flights Data
###############################################################################
dummy_flights_data = {
    "flights": [
        {
            "flight_number": "FA101",
            "origin": "NYC",
            "destination": "LAX",
            "departure_date": "2025-06-01",
            "departure_time": "08:00",
            "arrival_time": "11:00",
            "price": 320,
            "currency": "USD",
            "seat_availability": 60
        },
        {
            "flight_number": "FA102",
            "origin": "NYC",
            "destination": "LAX",
            "departure_date": "2025-06-02",
            "departure_time": "09:30",
            "arrival_time": "12:30",
            "price": 300,
            "currency": "USD",
            "seat_availability": 45
        },
        {
            "flight_number": "FA103",
            "origin": "NYC",
            "destination": "LAX",
            "departure_date": "2025-06-03",
            "departure_time": "14:00",
            "arrival_time": "17:00",
            "price": 340,
            "currency": "USD",
            "seat_availability": 80
        },
        {
            "flight_number": "FA104",
            "origin": "LAX",
            "destination": "NYC",
            "departure_date": "2025-06-01",
            "departure_time": "07:00",
            "arrival_time": "15:00",
            "price": 350,
            "currency": "USD",
            "seat_availability": 55
        },
        {
            "flight_number": "FA105",
            "origin": "LAX",
            "destination": "NYC",
            "departure_date": "2025-06-02",
            "departure_time": "12:00",
            "arrival_time": "20:00",
            "price": 330,
            "currency": "USD",
            "seat_availability": 72
        },
        {
            "flight_number": "FA106",
            "origin": "LAX",
            "destination": "NYC",
            "departure_date": "2025-06-03",
            "departure_time": "16:00",
            "arrival_time": "00:30",
            "price": 320,
            "currency": "USD",
            "seat_availability": 40
        }
    ]
}


In [12]:
###############################################################################
# 2) Helper Functions: route extraction, name extraction
###############################################################################
def extract_route_from_text(text):
    text = text.upper().strip(".,!?;'\" \t\n\r")
    pattern = r"FROM\s+([A-Z]{3})\s+TO\s+([A-Z]{3})"
    match = re.search(pattern, text)
    if match:
        return match.group(1), match.group(2)
    return None, None

def extract_name(text):
    text = text.strip()
    match = re.search(r"(?:my name is|i am|i'm)\s+([A-Z][a-zA-Z]+)", text, re.IGNORECASE)
    if match:
        return match.group(1).strip()
    words = text.split()
    if len(words) == 1:
        return words[0]
    return None

In [13]:
###############################################################################
# 3) Zero-shot Classifier (No LLMChain)
###############################################################################
classifier = pipeline("zero-shot-classification", model="valhalla/distilbart-mnli-12-3")

def classify_intent(text):
    text_lower = text.lower().strip(".,!?;'\" \t\n\r")

    if ("from" in text_lower and "to" in text_lower) or ("->" in text_lower):
        return "flight_info"
    if "cancel" in text_lower and ("flight" in text_lower or "booking" in text_lower):
        return "cancel_flight"
    if "reschedul" in text_lower and "flight" in text_lower:
        return "reschedule_flight"
    if "book" in text_lower and "flight" in text_lower:
        return "book_flight"
    if (("which" in text_lower or "list" in text_lower or "show" in text_lower)
        and ("booked" in text_lower or "bookings" in text_lower or "booking" in text_lower)):
        return "list_bookings"
    if ("no thanks" in text_lower or "no thank you" in text_lower or
        "bye" in text_lower or "i'm done" in text_lower):
        return "exit"

    labels = ["greet", "book_flight", "flight_info", "user_info", "exit", "fallback"]
    try:
        result = classifier(text, labels)
        return result["labels"][0]
    except:
        return "fallback"

Device set to use cuda:0


In [14]:
###############################################################################
# 4) Global State
###############################################################################
conversation_ended = False
bookings = []
user_info = {
    "name": None,                # main contact name
    "email": None,               # main contact email
    "flight_choice": None,
    "available_flight_options": [],
    "passenger_count": 1,
    "passenger_names": [],
    "baggage_count": 0,
    "final_cost": 0.0
}
next_action = None

In [15]:
###############################################################################
# 5) Payment & Confirmation Helpers
###############################################################################
import re

def handle_payment_step(user_message):
    global next_action, bookings, user_info
    card = user_message.strip().replace(" ", "")
    if re.match(r"^\d{16}$", card):
        chosen_flight = user_info["flight_choice"]
        price_charged = user_info["final_cost"]
        booking_record = {
            "flight_number": chosen_flight["flight_number"],
            "user_name": user_info["name"],
            "email": user_info["email"],
            "price": price_charged,
            "date": chosen_flight["departure_date"],
            "origin": chosen_flight["origin"],
            "destination": chosen_flight["destination"],
            "passenger_count": user_info["passenger_count"],
            "passenger_names": user_info["passenger_names"],
            "baggage_count": user_info["baggage_count"]
        }
        bookings.append(booking_record)

        next_action = None
        return (
            f"Payment successful! We have charged ${price_charged} to your card.\n"
            f"Booking confirmed for {user_info['name']}!\n"
            f"Flight: {chosen_flight['flight_number']} on {chosen_flight['departure_date']} "
            f"({chosen_flight['origin']}->{chosen_flight['destination']})\n"
            f"Total passengers: {user_info['passenger_count']} (names: {', '.join(user_info['passenger_names'])})\n"
            f"Extra checked bags: {user_info['baggage_count']}\n"
            f"Email: {user_info['email']}\n"
            "You're all set! Anything else I can help you with?"
        )
    else:
        return "That doesn't look like a valid 16-digit card number. Please try again."

def summarize_booking():
    chosen = user_info["flight_choice"]
    pcount = user_info["passenger_count"]
    names_str = ", ".join(user_info["passenger_names"])
    flight_info = (
        f"{chosen['flight_number']} on {chosen['departure_date']} "
        f"({chosen['origin']}->{chosen['destination']})"
    )
    extra_bags = user_info["baggage_count"]
    final_cost = user_info["final_cost"]
    return (
        "Please review your booking details:\n"
        f"Flight: {flight_info}\n"
        f"Main contact: {user_info['name']} (email: {user_info['email']})\n"
        f"Passengers: {pcount}\n"
        f"Passenger names: {names_str}\n"
        f"Extra checked bags: {extra_bags}\n"
        f"Total cost: ${final_cost}\n"
        "Do you confirm this booking? (yes/no)"
    )


In [16]:
###############################################################################
# 6) Main Bot Logic (Now we ask for name+email after flight is chosen)
###############################################################################
def process_user_input(user_message):
    global conversation_ended, next_action, user_info, bookings

    if conversation_ended:
        return "Conversation closed. Please refresh to start a new one."

    user_message = user_message.strip(".,!?;'\" \t\n\r")

    # A) Payment step
    if next_action == "get_payment":
        return handle_payment_step(user_message)

    # B) Final confirmation
    if next_action == "confirm_booking":
        msg_lower = user_message.lower()
        if "yes" in msg_lower:
            next_action = "get_payment"
            return "Great! Please provide your 16-digit credit card number to complete payment."
        else:
            # user declined => start route selection again
            next_action = "get_flight_choice"
            return "Okay, let's start over. Which route would you like to book? (e.g. 'NYC -> LAX')"

    # C) If we are collecting main contact name
    if next_action == "get_main_contact":
        # parse name
        parsed_name = extract_name(user_message)
        if parsed_name:
            user_info["name"] = parsed_name
        else:
            user_info["name"] = user_message.strip()
        next_action = "get_email"
        return f"Thanks, {user_info['name']}! Please provide your email."

    # D) If we are collecting main contact email
    if next_action == "get_email":
        # just store whatever user typed
        user_info["email"] = user_message.strip()
        # proceed to passenger_count
        next_action = "get_passenger_count"
        return (
            "Got it. Now, how many passengers in total (including you)?"
        )

    # E) If we are collecting passenger count
    if next_action == "get_passenger_count":
        if user_message.isdigit():
            count = int(user_message)
            if count < 1:
                count = 1
            user_info["passenger_count"] = count
            user_info["passenger_names"] = []
            if count == 1:
                # If there's only 1 passenger, that's the main contact
                user_info["passenger_names"].append(user_info["name"])
                next_action = "get_baggage"
                return (
                    f"Got it. {count} passenger total.\n"
                    "Each passenger can bring one personal item + one carry-on for free.\n"
                    "Any additional checked bag costs $40. How many extra bags in total?"
                )
            else:
                # If multiple, we ask for each passenger name
                next_action = "get_passenger_names"
                return (
                    f"Please provide the names of the {count} passengers, one by one.\n"
                    "What is the first passenger's name?"
                )
        else:
            return "Please enter a valid number of passengers."

    # F) If collecting multiple passenger names
    if next_action == "get_passenger_names":
        parsed_name = extract_name(user_message)
        if parsed_name:
            user_info["passenger_names"].append(parsed_name)
        else:
            user_info["passenger_names"].append(user_message.strip())

        if len(user_info["passenger_names"]) < user_info["passenger_count"]:
            return "Thanks. Next passenger's name?"
        else:
            # done collecting passenger names
            next_action = "get_baggage"
            return (
                f"Great, got all {user_info['passenger_count']} names.\n"
                "Each passenger gets 1 personal item + 1 carry-on free.\n"
                "Any extra checked bag costs $40. How many total extra bags?"
            )

    # G) If collecting baggage
    if next_action == "get_baggage":
        if user_message.isdigit():
            extra_bags = int(user_message)
            if extra_bags < 0:
                extra_bags = 0
            user_info["baggage_count"] = extra_bags

            chosen = user_info["flight_choice"]
            pcount = user_info["passenger_count"]
            base_price = chosen["price"] * pcount
            bag_cost = extra_bags * 40
            total = base_price + bag_cost
            user_info["final_cost"] = total

            next_action = "confirm_booking"
            return summarize_booking()
        else:
            return "Please enter a valid number for extra bags."

    # H) If we are waiting for flight number (booking)
    if next_action == "get_flight_number":
        flight_number = user_message.upper()
        chosen = next(
            (f for f in user_info["available_flight_options"]
             if f["flight_number"] == flight_number),
            None
        )
        if chosen:
            user_info["flight_choice"] = chosen
            # Now we ask for main contact name
            next_action = "get_main_contact"
            return (
                f"Great choice! You chose {flight_number}.\n"
                "May I have your name (main contact)?"
            )
        else:
            return "I couldn't find that flight number. Please try again."

    # I) If we are waiting for flight number (cancellation)
    if next_action == "get_cancellation_flight":
        flight_number = user_message.strip().upper()
        booking_to_cancel = next(
            (b for b in bookings if b["flight_number"] == flight_number),
            None
        )
        if booking_to_cancel:
            bookings.remove(booking_to_cancel)
            refund_amount = booking_to_cancel["price"]
            next_action = None
            return (
                f"Your booking for flight {flight_number} has been canceled.\n"
                f"A refund of ${refund_amount} will be processed.\n"
                "Anything else I can do?"
            )
        else:
            # user might say "which flights do I have booked?" or something else
            text_lower = user_message.lower()
            if ("which" in text_lower or "what" in text_lower or "list" in text_lower):
                user_intent_temp = classify_intent(text_lower)
                if user_intent_temp == "list_bookings":
                    if not bookings:
                        return "You have no bookings on record."
                    else:
                        listing = "\n".join(
                            f"{b['flight_number']} on {b['date']} "
                            f"({b['origin']}->{b['destination']})"
                            for b in bookings
                        )
                        return (
                            "Here are the flights you have booked:\n"
                            + listing
                            + "\n\nWhich one would you like to cancel?"
                        )
                else:
                    return (
                        "I couldn't find that booking in your record. "
                        "If you have multiple bookings, ask 'Which flights do I have booked?' "
                        "or type the flight number you want to cancel."
                    )
            else:
                return (
                    "I couldn't find that booking in your record. "
                    "Please type the flight number you want to cancel, or ask me which you have booked."
                )

    # J) If we are waiting for route
    if next_action == "get_flight_choice":
        origin, dest = extract_route_from_text(user_message)
        if not origin or not dest:
            parts = user_message.upper().split("->")
            if len(parts) == 2:
                origin, dest = parts[0].strip(), parts[1].strip()

        if origin and dest:
            flights = [
                f for f in dummy_flights_data["flights"]
                if f["origin"] == origin and f["destination"] == dest
            ]
            if flights:
                user_info["available_flight_options"] = flights
                flights_list = "\n".join([
                    f"{f['flight_number']} on {f['departure_date']} "
                    f"at {f['departure_time']} - ${f['price']}"
                    for f in flights
                ])
                next_action = "get_flight_number"
                return (
                    "Here are your flight options:\n"
                    + flights_list
                    + "\n\nWhich flight number would you like to book?"
                )
            else:
                next_action = None
                return "No flights found for that route."
        else:
            return "Please specify a route, e.g. 'I want to fly from NYC to LAX.'"

    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # K) Otherwise do classification
    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    intent = classify_intent(user_message)

    if intent == "exit":
        conversation_ended = True
        return "Thanks for flying with Fictitious Air. Goodbye!"

    if intent == "book_flight":
        next_action = "get_flight_choice"
        return "Sure! Which route would you like to book? (e.g. 'from NYC to LAX')"

    if intent == "flight_info":
        next_action = "get_flight_choice"
        return "Which route do you want info for? e.g. 'NYC -> LAX' or 'from NYC to LAX.'"

    if intent == "cancel_flight":
        next_action = "get_cancellation_flight"
        return "Sure, which flight number would you like to cancel?"

    if intent == "reschedule_flight":
        next_action = "get_reschedule_flight"
        return "Sure, which flight number do you want to reschedule?"

    if intent == "list_bookings":
        if not bookings:
            return "You have no bookings on record."
        else:
            listing = "\n".join(
                f"{b['flight_number']} on {b['date']} "
                f"({b['origin']}->{b['destination']}), "
                f"{b['passenger_count']} passengers, "
                f"${b['price']} total"
                for b in bookings
            )
            return "Here are your current bookings:\n" + listing

    if intent == "fallback":
        return "Sorry, I didn’t quite catch that. Can you rephrase?"

    # default fallback
    return (
        "No LLM is available to handle that request. "
        "Please try something else or type 'help' for assistance."
    )



In [17]:
###############################################################################
# 7) Gradio UI
###############################################################################
def chat_with_bot(message, history):
    reply = process_user_input(message)
    history.append((message, reply))
    return history, ""

initial_bot_message = (
    "Hello! How can I assist you today? "
    "You can book, cancel, or reschedule flights, list your bookings, or update info."
)

import gradio as gr

with gr.Blocks() as demo:
    gr.Markdown("## ✈️ Fictitious Air Chatbot (Fixed: Always ask for name & email)")
    chatbot = gr.Chatbot(value=[[None, initial_bot_message]])
    user_input = gr.Textbox(placeholder="Ask me about flights...", show_label=False)
    clear_btn = gr.Button("Clear")

    user_input.submit(chat_with_bot, [user_input, chatbot], [chatbot, user_input])
    clear_btn.click(lambda: ([[None, initial_bot_message]], ""), None, [chatbot, user_input])

demo.launch(inline=True)


<ipython-input-17-9e9ca8d6a3dd>:18: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(value=[[None, initial_bot_message]])


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d9164bc56a96cf163a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
